# 10. American Community Survey 5 Year Data

Switching gears to another dataset! The ultimate goal is to get familiar with this popular dataset and learn how to plot it to show information clearly.


**INSTRUCTOR NOTES**:
- Datasets used:
    - "../notebook_data/census/ACS5yr/census_variables_CA.csv"
    
- Expected time to complete:
    - Lecture + Questions: 15 minutes
    - Challenges: 5 minutes

### Import Packages

In [1]:
import math
import numpy as np
import pandas as pd

import json # for working with JSON data
import geojson # ditto for GeoJSON data - an extension of JSON with support for geographic data
import geopandas as gpd

import matplotlib # base python plotting library
%matplotlib inline  
import matplotlib.pyplot as plt # more plotting stuff


## 10.1 American Community Survey 5 Year Data (or ACS5)

To get started, let's read the ACS 5 year data for California tracts into a `dataframe` using the  `pandas read_csv` method. 

As we read in the ACS data we will tell pandas to make sure that the data in the column `FIPS_11_digit` is read in as a string to preserve leading zeros in the census tract identifiers.

In [2]:
# Read in the ACS5 data for CA into a pandas DataFrame.
# Note: We force the FIPS_11_digit to be read in as a string to preserve any leading zeroes.
acs5data_df = pd.read_csv("../notebook_data/census/ACS5yr/census_variables_CA.csv", dtype={'FIPS_11_digit':str})

Pandas provides a number of methods to view information about a dataframe.

The pandas dataframe attribute `shape` tells us the number of rows and columns in the dataframe.

In [3]:
# Take a look at the shape of the dataframe
acs5data_df.shape

(16114, 66)

Each row in our dataframe is an observation. For the ACS5 data each observation is about a census tract.

Each column in our dataframe is a variable for that observation.

Let's use `head` to take a look at the first 5 rows in the dataframe.

In [4]:
# Take a look at the data
acs5data_df.head()

,NAME,c_race,c_white,c_black,c_asian,c_latinx,c_race_moe,c_white_moe,c_black_moe,c_asian_moe,...,p_stay,p_movelocal,p_movecounty,p_movestate,p_moveabroad,p_car,p_carpool,p_transit,p_bike,p_walk
0,"Census Tract 4012, Alameda County, California",2456,1287,476,259,283,213,191,116,124,...,0.814951,0.103350,0.058415,0.010212,0.013072,0.551370,0.064384,0.189041,0.083562,0.058219
1,"Census Tract 4013, Alameda County, California",3983,845,1348,827,796,680,186,411,283,...,0.611865,0.280040,0.063348,0.022624,0.022122,0.341153,0.108993,0.391496,0.018084,0.104594
2,"Census Tract 4014, Alameda County, California",4340,713,1902,593,981,644,314,440,198,...,0.807683,0.163739,0.017803,0.006325,0.004451,0.470846,0.021317,0.255799,0.116614,0.102194
3,"Census Tract 4015, Alameda County, California",2080,563,1064,215,190,369,222,283,116,...,0.841346,0.101442,0.053846,0.003365,0.000000,0.502037,0.090631,0.230143,0.047862,0.017312
4,"Census Tract 4016, Alameda County, California",1889,324,960,247,274,400,135,376,164,...,0.830645,0.079570,0.082258,0.002151,0.005376,0.570481,0.122720,0.177446,0.063018,0.000000


A `...` in the middle of the top row indicates that there are two many columns to display.

The pandas dataframe `columns` attribute returns a list of the column names.

In [5]:
acs5data_df.columns

Index(['NAME', 'c_race', 'c_white', 'c_black', 'c_asian', 'c_latinx',
       'c_race_moe', 'c_white_moe', 'c_black_moe', 'c_asian_moe',
       'c_latinx_moe', 'state_fips', 'county_fips', 'tract_fips', 'med_rent',
       'med_hhinc', 'c_tenants', 'c_owners', 'c_renters', 'med_rent_moe',
       'med_hhinc_moe', 'c_tenants_moe', 'c_owners_moe', 'c_renters_moe',
       'c_movers', 'c_stay', 'c_movelocal', 'c_movecounty', 'c_movestate',
       'c_moveabroad', 'c_movers_moe', 'c_stay_moe', 'c_movelocal_moe',
       'c_movecounty_moe', 'c_movestate_moe', 'c_moveabroad_moe', 'c_commute',
       'c_car', 'c_carpool', 'c_transit', 'c_bike', 'c_walk', 'c_commute_moe',
       'c_car_moe', 'c_carpool_moe', 'c_transit_moe', 'c_bike_moe',
       'c_walk_moe', 'year', 'FIPS_11_digit', 'p_white', 'p_black', 'p_asian',
       'p_latinx', 'p_owners', 'p_renters', 'p_stay', 'p_movelocal',
       'p_movecounty', 'p_movestate', 'p_moveabroad', 'p_car', 'p_carpool',
       'p_transit', 'p_bike', 'p_walk'],


We can see more information about the variables included in our ACS5 year data using the `info` method. This method tells us at a glance what variables (or columns) are included in the data, the data type of each variable, and which variables have values for all rows.

In [6]:
acs5data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16114 entries, 0 to 16113
Data columns (total 66 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NAME              16114 non-null  object 
 1   c_race            16114 non-null  int64  
 2   c_white           16114 non-null  int64  
 3   c_black           16114 non-null  int64  
 4   c_asian           16114 non-null  int64  
 5   c_latinx          16114 non-null  int64  
 6   c_race_moe        16114 non-null  int64  
 7   c_white_moe       16114 non-null  int64  
 8   c_black_moe       16114 non-null  int64  
 9   c_asian_moe       16114 non-null  int64  
 10  c_latinx_moe      16114 non-null  int64  
 11  state_fips        16114 non-null  int64  
 12  county_fips       16114 non-null  int64  
 13  tract_fips        16114 non-null  int64  
 14  med_rent          15838 non-null  float64
 15  med_hhinc         15935 non-null  float64
 16  c_tenants         16114 non-null  int64 

### What's in the ACS data?

These variables were combined from different ACS 5 year tables. We have information for the following:

- `c_race` - Total population
- `c_white` - Total white non-Latinx
- `c_black` - Total black and African American non-Latinx
- `c_asian` - Total Asian non-Latinx
- `c_latinx` - Total Latinx
- `state_fips` - State level FIPS code
- `county_fips` - County level FIPS code
- `tract_fips` - Tracts level FIPS code
- `med_rent` - Median rent
- `med_hhinc` - Median household income
- `c_tenants` - Total tenants
- `c_owners` - Total owners
- `c_renters` - Total renters
- `c_movers` - Total number of people who moved
- `c_stay` - Total number of people who stayed
- `c_movelocal` - Number of people who moved locally
- `c_movecounty` - Number of people who moved counties
- `c_movestate` - Number of people who moved states
- `c_moveabroad` - Number of people who moved abroad
- `c_commute` - Total number of commuters
- `c_car` - Number of commuters who use a car
- `c_carpool` - Number of commuters who carpool
- `c_transit` - Number of commuters who use public transit
- `c_bike` - Number of commuters who bike
- `c_walk` - Number of commuters who bike
- `year` - ACS data year
- `FIPS_11_digit` - 11-digit FIPS code

The ACS variables that start with `c_` are counts, those that start with `med_` are medians.  Variables that end in `_moe` denote margin of error. There are also a number of derived variables that start with `p_`. These are proportions calcuated from the counts divided by the table denominator (the total count for whom that variable was assessed).

We're going to drop all of our `moe` columns by identifying all of those that end with `_moe`. We can do that in two steps, first by using `filter` to identify columns that contain the string `_moe`.

In [7]:
moe_cols = acs5data_df.filter(like='_moe',axis=1).columns
moe_cols

Index(['c_race_moe', 'c_white_moe', 'c_black_moe', 'c_asian_moe',
       'c_latinx_moe', 'med_rent_moe', 'med_hhinc_moe', 'c_tenants_moe',
       'c_owners_moe', 'c_renters_moe', 'c_movers_moe', 'c_stay_moe',
       'c_movelocal_moe', 'c_movecounty_moe', 'c_movestate_moe',
       'c_moveabroad_moe', 'c_commute_moe', 'c_car_moe', 'c_carpool_moe',
       'c_transit_moe', 'c_bike_moe', 'c_walk_moe'],
      dtype='object')

Note how we set the filter `like=` to a value that matches the pattern of the names of the columns we want to drop. You need to make sure you get all but only the columns that you want to drop.

<div style="display:inline-block;vertical-align:top;">
    <img src="http://www.pngall.com/wp-content/uploads/2016/03/Light-Bulb-Free-PNG-Image.png" width="30" align=left > 
</div>  
<div style="display:inline-block;">

#### Question
</div>

What do you think happens if you match `_mo` instead of `_moe` in the filter?

Now that we've got our list of moe columns, we can use `.drop()` to remove them from the dataframe. 

In [8]:
# Drop MOE columns
acs5data_df.drop(moe_cols, axis=1, inplace=True)

Check that you no longer have the moe columns in the dataframe.

In [9]:
acs5data_df.columns

Index(['NAME', 'c_race', 'c_white', 'c_black', 'c_asian', 'c_latinx',
       'state_fips', 'county_fips', 'tract_fips', 'med_rent', 'med_hhinc',
       'c_tenants', 'c_owners', 'c_renters', 'c_movers', 'c_stay',
       'c_movelocal', 'c_movecounty', 'c_movestate', 'c_moveabroad',
       'c_commute', 'c_car', 'c_carpool', 'c_transit', 'c_bike', 'c_walk',
       'year', 'FIPS_11_digit', 'p_white', 'p_black', 'p_asian', 'p_latinx',
       'p_owners', 'p_renters', 'p_stay', 'p_movelocal', 'p_movecounty',
       'p_movestate', 'p_moveabroad', 'p_car', 'p_carpool', 'p_transit',
       'p_bike', 'p_walk'],
      dtype='object')

## 10.2 Select data for our county and year of interest

Our ACS5 data contains observations for all CA counties and two ACS 5 year periods.

The counties are identified by a unique Census FIPS code. 
- You can see the list of all CA Counties and their FIPS codes [here](https://en.wikipedia.org/wiki/List_of_counties_in_California).

Let's use the `.unique` to check the unique set of county FIPS codes included in our dataframe.

In [10]:
acs5data_df['county_fips'].unique()  #what counties are in our dataframe

array([  1,  13,   3,   5,   7,   9,  11,  29,  31,  33,  35,  37,  15,
        17,  19,  21,  23,  25,  27,  39,  41,  43,  45,  47,  49,  51,
        53,  55,  57,  59,  65,  61,  63,  67,  71,  69,  73,  79,  81,
        75,  77,  85,  95,  97,  87,  89,  91,  93,  83, 101, 103, 105,
       107, 109, 111, 113, 115,  99])

Now use `.unique` to see what years are included.

In [11]:
acs5data_df['year'].unique()

array([2013, 2018])

We are interested in Alameda County, which has the FIPS code `001`.  Moreover, we are only interested in the 2018 ACS 5 year data.  Let's filter the data to keep only the rows that match these two conditions.


In [12]:
acs5data_df_ac = acs5data_df[(acs5data_df['year']==2018) & (acs5data_df['county_fips']==1)]

<div style="display:inline-block;vertical-align:top;">
    <img src="http://www.pngall.com/wp-content/uploads/2016/03/Light-Bulb-Free-PNG-Image.png" width="30" align=left > 
</div>  
<div style="display:inline-block;">

#### Question
</div>

Why do we filter on `county_fips==1` instead of `county_fips==001` or `county_fips=='001'`?

In [13]:
# Write your thoughts here

Now, check the contents of our dataframe again.

In [14]:
# now what is the shape of the data when filtered for Alameda County?
print(acs5data_df_ac.shape)

(361, 44)


In [15]:
# Take a look at the first 5 rows
acs5data_df_ac.head()

,NAME,c_race,c_white,c_black,c_asian,c_latinx,state_fips,county_fips,tract_fips,med_rent,...,p_stay,p_movelocal,p_movecounty,p_movestate,p_moveabroad,p_car,p_carpool,p_transit,p_bike,p_walk
8323,"Census Tract 4415.01, Alameda County, California",6570,677,111,4740,570,6,1,441501,1883.0,...,0.925897,0.039593,0.010476,0.019874,0.004160,0.761761,0.113940,0.054812,0.012085,0.003453
8324,"Census Tract 4047, Alameda County, California",2079,1515,134,199,175,6,1,404700,3250.0,...,0.891826,0.028390,0.037690,0.031816,0.010279,0.532093,0.177674,0.158140,0.006512,0.005581
8325,"Census Tract 4425, Alameda County, California",7748,1430,375,3379,1904,6,1,442500,1999.0,...,0.899056,0.073020,0.003802,0.003015,0.021106,0.715315,0.112613,0.064264,0.017718,0.010210
8326,"Census Tract 4503, Alameda County, California",5301,2597,96,1077,1315,6,1,450300,2626.0,...,0.889312,0.025529,0.062107,0.021718,0.001334,0.749202,0.064917,0.094005,0.009933,0.011706
8327,"Census Tract 4506.07, Alameda County, California",5971,2832,324,1726,804,6,1,450607,1837.0,...,0.801863,0.121084,0.043014,0.034039,0.000000,0.698643,0.132730,0.058220,0.021418,0.034389


>**Pro-tip:** Checking your row and column counts and values often with `.shape` and values with `.head` help to make sure that these values are consistent with your understanding of the data.

## 10.3 Saving our output

It's a good idea to save your data if you have done any major processing on it. Let's save our Alameda County sub-setted ACS5 data to a CSV file.

In [16]:
# Save processed data to a csv file - give it a name that is meaningful
acs5data_df_ac.to_csv('../outdata/acs5data_2018_AC.csv')

Confirm that the file was saved with a [shell command](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html#Shell-Commands-in-IPython).  Shell commands are prefaced by a `!` and allow you to access the file system and run commands like you would from a terminal window. (This may differ if you are on a windows computer)

In [17]:
!ls ../outdata

acs5data_2018_AC.csv              berkeley_parcels_gdf.json
bart_buffers.json                 berkeley_parcels_gdf.prj
berkeley_parcels_gdf.cpg          berkeley_parcels_gdf.shp
berkeley_parcels_gdf.csv          berkeley_parcels_gdf.shx
berkeley_parcels_gdf.dbf          berkeley_parcels_gdf_limited.json
berkeley_parcels_gdf.gpkg         berkeley_schools.json



<div style="display:inline-block;vertical-align:top;">
    <img src="../../assets/images/E57A06C8-033C-4EF6-AB00-9FE1736F74C0_1_105_c.jpeg" align=center width=600 > 
</div>  
<div style="display:inline-block;">
    
    
<br>
    
    
## 10.4 Challenge

Now do this for the SF ACS data:
1. Find the FIPS code for [SF county](https://en.wikipedia.org/wiki/List_of_counties_in_California)
2. Subset the ACS data to keep only rows for SF county in 2018 and assign to `acs5data_df_sf`
3. Save out ACS data as `acs5data_2018_SF.csv`




In [18]:
# Your code here


*Click here for solution*

<!--- 
    # SOLUTION
    # 1 & 2 Subset ACS data for SF
    acs5data_df_sf = acs5data_df[(acs5data_df['county_fips']==75) & (acs5data_df.year==2018)]

    # SOLUTION
    acs5data_df_sf.head()

    # SOLUTION
    # 3. Save out ACS data as 'acs5data_2018_SF.csv'
    acs5data_df_sf.to_csv('../outdata/acs5data_2018_SF.csv')
--->

---
<div style="display:inline-block;vertical-align:middle;">
<a href="https://dlab.berkeley.edu/" target="_blank"><img src ="../../assets/images/dlab_logo.png" width="75" align="left">
</a>
</div>

<div style="display:inline-block;vertical-align:middle;">
    <div style="font-size:larger">&nbsp;D-Lab @ University of California - Berkeley</div>
    <div>&nbsp;Team Geo<div>
</div>